In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix

GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 50#10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
WB = 110#2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 100#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -100#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True

def pi_2_pi(angle):
    while(angle > math.pi):
        angle = angle - 2.0 * math.pi

    while(angle < -math.pi):
        angle = angle + 2.0 * math.pi

    return angle

def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False

def smooth_yaw(yaw):

    for i in range(len(yaw) - 1):
        dyaw = yaw[i + 1] - yaw[i]

        while dyaw >= math.pi / 2.0:
            yaw[i + 1] -= math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

        while dyaw <= -math.pi / 2.0:
            yaw[i + 1] += math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

    return yaw


In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od,x0):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref,x0)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")
    
    return oa, od, ox, oy, oyaw, ov,xbar

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref,x0):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = np.array(x.value[0, :]).flatten()
        oy = np.array(x.value[1, :]).flatten()
        ov = np.array(x.value[2, :]).flatten()
        oyaw = np.array(x.value[3, :]).flatten()
        oa = np.array(u.value[0, :]).flatten()
        odelta = np.array(u.value[1, :]).flatten()

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta):

        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = self.state.x + self.state.v * math.cos(self.state.yaw) * DT
        next_y = self.state.y + self.state.v * math.sin(self.state.yaw) * DT
        next_yaw = self.state.yaw + self.state.v / WB * math.tan(delta) * DT
        next_v = self.state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw

        return xbar


In [4]:
def gen_points(cx,cy):
    dx=20#2
    dy=10#1
    pts=[[cx-dx,cy+dy],
    [cx+dx,cy+dy],
    [cx+dx,cy-dy],
    [cx-dx,cy-dy],
    [cx-dx,cy+dy]]
    pts=np.array(pts,dtype=np.float64)
    return pts
class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points

In [5]:
%matplotlib tk
import gym
from matplotlib import pyplot as plt
import cv2
import numpy as np
from car_env import CarRacing
import time
import cubic_spline_planner
def rad_to_deg():
    pass
def accl_to_gas(accl,vel):
    SIZE = 0.02
    ENGINE_POWER= 100000000*SIZE*SIZE
    WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
    WHEEL_R  = 27 
    wheel_rad=WHEEL_R*SIZE
    mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
    #mass=.25
    power=4*mass*accl*(vel+accl/2)
    gas=power/ENGINE_POWER
    return gas

def env_step(env,action,show_plot=False):
    
    frame=np.zeros((100,200,3),dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    #action[0]=action[0]-np.deg2rad(90)
    action[0]*=-1
    nst,rw,ter,info=envc.step(action) # take a random action
    frame[0:96,0:96,:]=np.array(nst)
    
    pos=envc.car.hull.position
    vel=np.sqrt(envc.car.hull.linearVelocity[0]**2+envc.car.hull.linearVelocity[1]**2)
    ang=envc.car.hull.angle
    #ang*=-1
    ang=ang+np.deg2rad(90)#-ang
    if show_plot==True:
        plt.cla()
        img = cv2.putText(np.array(frame), 'vel %f'%vel, (100,10), font,.3, (255,255,0),1, cv2.LINE_AA)
        img = cv2.putText(np.array(img), 'ang %f'%ang, (100,20), font,.3, (255,0,255),1, cv2.LINE_AA)
        image = cv2.putText(np.array(img), 'x,y %f,%f'%(pos[0],pos[1]), (100,30), font,.2, (255,0,255),1, cv2.LINE_AA)
       
        plt.imshow(image)
        plt.show()
        plt.pause(0.0001)
        
    return [pos[0],pos[1],ang,vel],rw,ter,info
#envc.close()


envc=CarRacing()
def env_reset(envc):
    import pickle
    envc.reset()
    save=False
    if save==True:
        f=open('path-1','wb')
        pickle.dump(envc.track,f)
        f.close()
    f=open('path-1','rb')
    track=pickle.load(f)
    f.close()
    envc.track=track
    
    points=[[x,y] for a,b,x,y in envc.track]
    points=np.array(points)
    plt.plot(points[:,0],points[:,1])
    plt.show()
    return [env_step(envc,[0,0,0]),points]
    
#st,points=env_reset(envc)
'''
for i in range(100):
    st,rw,ter,info=env_step(envc,[0,1,0])
    time.sleep(0.1)
    #print(st)
    '''
#envc.close()



'\nfor i in range(100):\n    st,rw,ter,info=env_step(envc,[0,1,0])\n    time.sleep(0.1)\n    #print(st)\n    '

In [6]:
def calc_speed_profile(cx, cy, cyaw, target_speed):

    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile

def get_closest_point(state, traj_points,pind):
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy, _, _) in traj_points]
    mind = min(d)
    ind = d.index(mind) + pind
    return ind

def calc_ref_trajectory(state, traj_points,dl):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    xref[:, 0] = traj_points[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(traj_points)-1:
            xref[:, i] = traj_points[i]
            dref[0, i] = 0.0
        else:
            xref[:, i] = traj_points[-1]
            dref[0, i] = 0.0

    return xref,dref

In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
envc=CarRacing()
[init_state,traj_points]=env_reset(envc)
ix,iy,iyaw,ivel=init_state[0]
tx,ty=traj_points[:50,0],traj_points[:50,1]

cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
#yaw0=np.rad2deg(90)
#cyaw=[yaw-yaw0 for yaw in cyaw]
#dcx=np.gradient(cx)
#dcy=np.gradient(cy)
#cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]
sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)


Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
print(state)

Track generation: 1121..1405 -> 284-tiles track


In [8]:
# initial yaw compensation
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
Motion_Status.set_state(state)
goal = [cx[-1], cy[-1]]
from collections import deque
class data_stream():
    def __init__(self,cx,cy,sp,cyaw,qlen):
        self.point_stream=deque(maxlen=qlen)
        self.x=cx
        self.y=cy
        self.v=sp
        self.yaw=cyaw
        self.index=0
        for _ in range(qlen):
            ind=self.index
            self.point_stream.append([self.x[ind],self.y[ind],self.v[ind],self.yaw[ind]])
            self.index+=1
        
    def get(self):
        return self.point_stream
    def add(self,num):
        for _ in range(num):
            ind=self.index
            self.point_stream.append([self.x[ind],self.y[ind],self.v[ind],self.yaw[ind]])
            self.index+=1
sensor_stream=data_stream(cx,cy,sp,cyaw,15)

In [9]:
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]
def get_raw_sensor_value():
    pass




#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
prev_index=0
cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''
traj_points=sensor_stream.get()
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        print("x,y,yaw,v\n",state.x,state.y,state.yaw,state.v)
        state_str=str(state.x)+','+str(state.y)+','+str(state.v)+','+str(np.rad2deg(state.yaw))+'\n'
        #state_str=str(state.yaw)+'\n'
        
        index= get_closest_point(state,traj_points, prev_index)
        diff=index-prev_index
        if diff>0:
            sensor_stream.add(diff)
            
        prev_index=index
        traj_points=sensor_stream.get()
        xref, dref = calc_ref_trajectory(state, traj_points, dl)
        
        print('xref...............\n',xref,'\n')
        x0 = [state.x, state.y, state.v, state.yaw]  # current state

        oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(
            xref, Motion_Status, dref, oa, odelta,x0)
        xref_str=str(xref[0][0])+','+str(xref[1][0])+','+str(xref[2][0])+','+str(np.rad2deg(xref[3][0]))+','+str(index)+'\n'
        xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        if odelta is not None:
            di, ai = odelta[0], oa[0]
        accl=ai
        gas=accl_to_gas(accl,state.v)
        steer=di
        print('accl,gas,steer\n',accl,gas,steer)
        act_str=str(accl)+','+str(gas)+','+str(np.rad2deg(steer))+'\n'
        #act_str=str(steer)+'\n'
        nst,rw,ter,info=env_step(envc,[steer,gas,0])
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        #next_state = Motion_Status.get_next_state(ai, di)
        Motion_Status.set_state(next_state)
        
        time = time + DT

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        if check_goal(state, goal, index, len(cx)):
            print("Goal")
            break

        if show_animation:  # pragma: no cover
            plt.cla()
            plt.text(50,50,'state : '+state_str,fontsize=12)
            plt.text(50,40,'xref : '+xref_str,fontsize=12)
            plt.text(50,30,'xbar : '+xbar_str,fontsize=12)
            plt.text(50,20,'act : '+act_str,fontsize=12)
            
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(x, y, "ob", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(cx[index], cy[index], "xg", label="target")
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            points=gen_points(state.x,state.y)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=move_points(points,0,0,ang_yaw)
            r_points=rp.apply()
            points=np.array(points,dtype=np.float64)
            r_points=np.array(r_points,dtype=np.float64)
            plt.plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            plt.title("Time[s]:" + str(round(time, 2))
                      + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
                      + '\nyaw'+str(state.yaw)
                      + '\nxref_yaw'+str(xref[3][0])
                      + '\nsteer'+str(steer))
            plt.pause(0.0001)

if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


x,y,yaw,v
 218.3634490966797 -51.68230056762695 1.4487485672023634 9.990239694972157e-07
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0.9999998955960124 0.0002962968264414122 -0.003070897700342991
214.3634490966797 -49.68230056762695
x,y,yaw,v
 218.36346435546875 -51.682159423828125 1.448763676979814 0.00718720583263402
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,stee

x,y,yaw,v
 218.36534118652344 -51.66706848144531 1.4487445736911635 0.09708148897275669
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0.9999974307310006 0.000353824397202132 -0.00977672279700221
214.36534118652344 -49.66706848144531
x,y,yaw,v
 218.3656005859375 -51.66500473022461 1.4487432772901396 0.10396380807533409
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 

x,y,yaw,v
 218.37124633789062 -51.6211051940918 1.4485948468234877 0.2069000178086674
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0.999998607116861 0.0004189027180675272 -0.01995330096685246
214.37124633789062 -49.6211051940918
x,y,yaw,v
 218.37181091308594 -51.61684799194336 1.4485744992878775 0.21473245911082292
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0.

x,y,yaw,v
 218.38221740722656 -51.54024887084961 1.4480353380229811 0.3312527772297097
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0.9999966017684114 0.0004925915575290333 -0.03447929941526458
214.38221740722656 -49.54024887084961
x,y,yaw,v
 218.3831787109375 -51.53351593017578 1.4479447389629225 0.3400543314308139
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0

x,y,yaw,v
 218.39976501464844 -51.42024230957031 1.4465257311847548 0.47048000227565373
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 0.9999965650838062 0.0005750962674405811 -0.06067793514751285
214.39976501464844 -49.42024230957031
x,y,yaw,v
 218.40122985839844 -51.41074752807617 1.446373381712709 0.4803025672573564
xref...............
 [[219.73870516 219.84009789 219.94149062 220.04288335 220.14427608
  220.24566881]
 [-52.03212331 -51.03727683 -50.04243035 -49.04758387 -48.0527374
  -47.05789092]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  1.46922906   1.46922906   1.46922906   1.46922906   1.46922906
    1.46922906]] 

accl,gas,steer
 

...

accl,gas,steer
 0.9999997749155297 0.009076477310899757 -0.2876285412019067
153.44264221191406 49.603179931640625
x,y,yaw,v
 157.15518188476562 47.67732238769531 2.714580730595861 14.816553202339295
xref...............
 [[157.22889556 156.34841525 155.46793493 154.58745461 144.90217112
  144.90217112]
 [ 47.47732502  47.95140773  48.42549043  48.89957313  54.11448288
   54.11448288]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 0.9999979306081291 0.009076456575819148 -0.7850870770501759
153.15518188476562 49.67732238769531
x,y,yaw,v
 156.86575317382812 47.750152587890625 2.7445880929615836 14.902465932283198
xref...............
 [[157.22889556 156.34841525 155.46793493 154.58745461 144.90217112
  144.90217112]
 [ 47.47732502  47.95140773  48.42549043  48.89957313  54.11448288
   54.11448288]
 [ 50.          50.          50.          50.          50.
   50. 

149.19171142578125 51.14621353149414
x,y,yaw,v
 152.9470977783203 49.320491790771484 2.703477100529943 14.99263502587402
xref...............
 [[152.82649398 151.94601366 151.06553334 150.18505303 140.49976953
  140.49976953]
 [ 49.84773854  50.32182125  50.79590395  51.26998666  56.4848964
   56.4848964 ]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 0.9999994142709218 0.009180815205050558 -0.7853737070120124
148.9470977783203 51.320491790771484
x,y,yaw,v
 152.70855712890625 49.50271224975586 2.6702374974873404 14.982682587639992
xref...............
 [[152.82649398 151.94601366 151.06553334 150.18505303 140.49976953
  140.49976953]
 [ 49.84773854  50.32182125  50.79590395  51.26998666  56.4848964
   56.4848964 ]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2

accl,gas,steer
 0.9999965091136522 0.009414223881731663 0.7853409502401993
145.34532165527344 54.14382553100586
x,y,yaw,v
 149.07049560546875 52.279544830322266 2.535197869935308 15.309774844617673
xref...............
 [[149.30457271 148.42409239 147.54361207 146.66313176 136.97784826
  136.97784826]
 [ 51.74406936  52.21815207  52.69223477  53.16631747  58.38122722
   58.38122722]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 0.9999983543263172 0.009368739557956618 0.785363613241425
145.07049560546875 54.279544830322266
x,y,yaw,v
 148.790283203125 52.39932632446289 2.5653746287014823 15.213656476162509
xref...............
 [[148.42409239 147.54361207 146.66313176 145.78265144 136.09736795
  136.09736795]
 [ 52.21815207  52.69223477  53.16631747  53.64040018  58.85530993
   58.85530993]
 [ 50.          50.          50.          50.          50.
   50.      

140.9724884033203 55.613929748535156
x,y,yaw,v
 144.68582153320312 53.73946762084961 2.809183911481176 15.64304789305332
xref...............
 [[144.90217112 144.0216908  143.14121049 142.26073017 132.57544668
  132.57544668]
 [ 54.11448288  54.58856559  55.06264829  55.536731    60.75164074
   60.75164074]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.64767073]] 

accl,gas,steer
 0.999998947130642 0.00956624021931776 -0.7853845090471069
140.68582153320312 55.73946762084961
x,y,yaw,v
 144.4051055908203 53.87609100341797 2.7931662360813956 15.602792467661432
xref...............
 [[144.90217112 144.0216908  143.14121049 142.26073017 132.57544668
  132.57544668]
 [ 54.11448288  54.58856559  55.06264829  55.536731    60.75164074
   60.75164074]
 [ 50.          50.          50.          50.          50.
   50.        ]
 [  2.64767073   2.64767073   2.64767073   2.64767073   2.64767073
    2.

IndexError: list index out of range

In [ ]:
_cx

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
i_yaw=cyaw[0]
cyaw=[yaw-i_yaw for yaw in cyaw]
cyaw

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
yaw0=np.deg2rad(90)
cyaw=[yaw-yaw0 for yaw in cyaw]

In [ ]:
import numpy as np
np.rad2deg(2.64)

In [ ]:
cyaw